In [ ]:
import os
import sys
import csv
import pvl

import pandas as pd
import numpy as np
import math
import pyproj
from functools import singledispatch
import warnings

from plio.examples import get_path
from plio.io.io_bae import read_gpf, read_ipf, read_atf
from plio.utils.utils import find_in_dict
from plio.spatial.transformations import *
from collections import defaultdict
import plio.io.io_controlnetwork as cn
import plio.io.isis_serial_number as sn

In [ ]:
def socet2isis(at_file, cub_file_path, cub_ipf_map, target_name, outpath=None):
    # Setup the at_file, path to cubes, and control network out path
    at_file = at_file
    cnet_out = os.path.split(os.path.splitext(at_file)[0])[1]
    cub_path = cub_file_path

    if(outpath):
        outpath = outpath
    else:
        outpath = os.path.split(at_file)[0]

    with open(cub_ipf_map) as cub_ipf_map:
        reader = csv.reader(cub_ipf_map, delimiter = ',')
        image_dict = dict([(row[0], row[1]) for row in reader])
        
    # Read in and setup the atf dict of information
    atf_dict = read_atf(at_file)
    
    # Get the gpf and ipf files using atf dict
    gpf_file = os.path.join(atf_dict['PATH'], atf_dict['GP_FILE']);
    ipf_list = [os.path.join(atf_dict['PATH'], i) for i in atf_dict['IMAGE_IPF']]
    
    # Read in the gpf file and ipf file(s) into seperate dataframes
    gpf_df = read_gpf(gpf_file)
    ipf_df = read_ipf(ipf_list)

    # Check for differences between point ids using each dataframes
    # point ids as a reference
    gpf_pt_idx = pd.Index(pd.unique(gpf_df['point_id']))
    ipf_pt_idx = pd.Index(pd.unique(ipf_df['pt_id']))

    point_diff = ipf_pt_idx.difference(gpf_pt_idx)

    if len(point_diff) != 0:
        warnings.warn("The following points found in ipf files missing from gpf file: \n\n{}. \
                      \n\nContinuing, but these points will be missing from the control network".format(list(point_diff)))
        
    # Merge the two dataframes on their point id columns
    socet_df = ipf_df.merge(gpf_df, left_on='pt_id', right_on='point_id')
    
    # Apply the transformations
    apply_transformations(atf_dict, socet_df)
    
    # Define column remap for socet dataframe
    column_map = {'pt_id': 'id', 'l.': 'y', 's.': 'x',
                  'res_l': 'lineResidual', 'res_s': 'sampleResidual', 'known': 'Type',
                  'lat_Y_North': 'aprioriY', 'long_X_East': 'aprioriX', 'ht': 'aprioriZ',
                  'sig0': 'aprioriLatitudeSigma', 'sig1': 'aprioriLongitudeSigma', 'sig2': 'aprioriRadiusSigma',
                  'sig_l': 'linesigma', 'sig_s': 'samplesigma'}
    
    # Rename the columns using the column remap above
    socet_df.rename(columns = column_map, inplace=True)
    
    serial_dict = serial_numbers(image_dict, cub_path)

    # creates the control network
    cn.to_isis(os.path.join(outpath, cnet_out + '.net'), socet_df, serial_dict, targetname = targetname)
    return socet_df

In [ ]:
# Setup stuffs for the cub information namely the path and extension
cub_path = '/Volumes/Blueman/'
targetname = 'Mars'
cub_map = '/Users/adampaquette/repos/plio/plio/examples/SocetSet/cub_map2.csv'

# Path to atf file
atf_file = ('/Users/adampaquette/repos/plio/plio/examples/SocetSet/Relative.atf')

socet_df = socet2isis(atf_file, cub_path, cub_map, targetname)

In [ ]:
def reverse_known(record):
    """
    Converts the known field from a socet dataframe into the
    isis point_type column

    Parameters
    ----------
    record : object
             Pandas series object

    Returns
    -------
    : str
      String representation of a known field
    """
    record_type = record['known']
    if record_type == 0 or record_type == 2:
        return 0

    elif record_type == 1 or record_type == 3 or record_type == 4:
        return 3
    
def lat_socet_coord(record, semi_major, semi_minor):
    """
    Function to convert lat_Y_North to ISIS_lat

    Parameters
    ----------
    record : object
             Pandas series object

    semi_major : float
                 Radius from the center of the body to the equater

    semi_minor : float
                 Radius from the pole to the center of mass

    Returns
    -------
    coord_360 : float
                Converted latitude into ocentric space, and mapped
                into 0 to 360
    """
    ographic_coord = oc2og(record['lat_Y_North'], semi_major, semi_minor)
    return ((ographic_coord + 180) % 360) - 180

def lon_socet_coord(record, semi_major, semi_minor):
    """
    Function to convert lat_Y_North to ISIS_lat

    Parameters
    ----------
    record : object
             Pandas series object

    semi_major : float
                 Radius from the center of the body to the equater

    semi_minor : float
                 Radius from the pole to the center of mass

    Returns
    -------
    coord_360 : float
                Converted latitude into ocentric space, and mapped
                into 0 to 360
    """
    ographic_coord = oc2og(record['long_X_East'], semi_major, semi_minor)
    return ((ographic_coord + 180) % 360) - 180

def fix_sample_line(record, serial_dict, extension, cub_path):
    # Cube location to load
    cube = pvl.load(os.path.join(cub_path, serial_dict[record['serialnumber']] + extension))
    line_size = find_in_dict(cube, 'Lines')
    sample_size = find_in_dict(cube, 'Samples')

    new_line = record['l.'] - (int(line_size)/2.0) - 1
    new_sample = record['s.'] - (int(sample_size)/2.0) - 1
    return new_line, new_sample

def ignore_toggle(record):
    if record['stat'] == True:
        return 0
    else:
        return 1

In [ ]:
return_df = cn.from_isis("/Users/adampaquette/repos/plio/plio/examples/SocetSet/Relative.net")

eRadius = 3.39619000000000e+006
pRadius = eRadius * (1 - 1.08339143554195e-001)

adjusted_flag = False

cub_path = '/Volumes/Blueman/'
extension = '.8bit.cub'
cub_list = ['D06_029601_1846_XN_04N224W', 
            'F05_037684_1857_XN_05N224W']

# 
cub_dict = {i: i + extension for i in cub_list}
serial_dict = {sn.generate_serial_number(os.path.join(cub_path, i + extension)): i for i in cub_list}

columns = []
column_index = []

for i, column in enumerate(list(return_df.columns)):
    if column not in columns:
        column_index.append(i)
        columns.append(column)

return_df = return_df.iloc[:, column_index]

column_map = {'id': 'pt_id', 'line': 'l.', 'sample': 's.', 
              'lineResidual': 'res_l', 'sampleResidual': 'res_s', 'type': 'known', 
              'aprioriLatitudeSigma': 'sig0', 'aprioriLongitudeSigma': 'sig1', 'aprioriRadiusSigma': 'sig2', 
              'linesigma': 'sig_l', 'samplesigma': 'sig_s', 'ignore': 'stat'}

if adjusted_flag:
    column_map['adjustedY'] = 'lat_Y_North'
    column_map['adjustedX'] = 'long_X_East'
    column_map['adjustedZ'] = 'ht'
else:
    column_map['aprioriY'] = 'lat_Y_North'
    column_map['aprioriX'] = 'long_X_East'
    column_map['aprioriZ'] = 'ht'

return_df.rename(columns = column_map, inplace=True)

return_df['known'] = return_df.apply(reverse_known, axis = 1)
return_df['ipf_file'] = return_df['serialnumber'].apply(lambda serial_number: serial_dict[serial_number])
return_df['l.'], return_df['s.'] = zip(*return_df.apply(fix_sample_line, serial_dict = serial_dict, 
                                                                         extension = extension, 
                                                                         cub_path = cub_path, axis = 1))

ecef = np.array([[return_df['long_X_East']], [return_df['lat_Y_North']], [return_df['ht']]])
lla = body_fix(ecef, semi_major = eRadius, semi_minor = pRadius, inverse=True)
return_df['long_X_East'], return_df['lat_Y_North'], return_df['ht'] = lla[0][0], lla[1][0], lla[2][0]

return_df['lat_Y_North'] = return_df.apply(lat_socet_coord, semi_major=eRadius, semi_minor=pRadius, axis = 1)
return_df['long_X_East'] = return_df.apply(lon_socet_coord, semi_major=eRadius, semi_minor=pRadius, axis = 1)

return_df['stat'] = return_df.apply(ignore_toggle, axis = 1)
return_df['val'] = return_df['stat']

# Add dumby
x_dummy = lambda x: np.full(len(return_df), x)

return_df['sig0'] = x_dummy(0)
return_df['sig1'] = x_dummy(0)
return_df['sig2'] = x_dummy(0)

return_df['res0'] = x_dummy(0)
return_df['res1'] = x_dummy(0)
return_df['res2'] = x_dummy(0)

return_df['fid_x'] = x_dummy(0)
return_df['fid_y'] = x_dummy(0)

return_df['no_obs'] = x_dummy(1)
return_df['fid_val'] = x_dummy(0)

In [ ]:
return_df[['long_X_East', 'lat_Y_North', 'ht']].iloc[2]

In [ ]:
return_df[['long_X_East', 'lat_Y_North', 'ht']].iloc[2]